In [42]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [43]:
data = open('flare').read().replace('\n', '').replace('\r', '')  # Load and clean the data
print(data)

flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained h

# 字符去重处理

In [44]:
letters = list(set(data))
num_letters = len(letters)
print(letters)
print(num_letters)

['d', 'c', 'p', 'l', 'r', ' ', 'f', 'n', '.', 'e', 'i', 'y', 't', 'a', 'H', 'A', 'o', 'S', 'm', 'h', 's', 'b', 'u']
23


# 构建字符到数字及数字到字符的映射字典

In [45]:
int_to_char = {a: b for a, b in enumerate(letters)}
char_to_int = {b: a for a, b in enumerate(letters)}
print(int_to_char)
print(char_to_int)

{0: 'd', 1: 'c', 2: 'p', 3: 'l', 4: 'r', 5: ' ', 6: 'f', 7: 'n', 8: '.', 9: 'e', 10: 'i', 11: 'y', 12: 't', 13: 'a', 14: 'H', 15: 'A', 16: 'o', 17: 'S', 18: 'm', 19: 'h', 20: 's', 21: 'b', 22: 'u'}
{'d': 0, 'c': 1, 'p': 2, 'l': 3, 'r': 4, ' ': 5, 'f': 6, 'n': 7, '.': 8, 'e': 9, 'i': 10, 'y': 11, 't': 12, 'a': 13, 'H': 14, 'A': 15, 'o': 16, 'S': 17, 'm': 18, 'h': 19, 's': 20, 'b': 21, 'u': 22}


In [46]:
time_step = 20

# 数据预处理

In [47]:
def extract_data(data, slide):
    x, y = [], []
    for i in range(len(data) - slide):
        x.append([a for a in data[i:i+slide]])
        y.append(data[i+slide])
    return x, y

In [48]:
def char_to_int_data(x, y, char_to_int):
    x_to_int = [[char_to_int[char] for char in sequence] for sequence in x]
    y_to_int = [char_to_int[char] for char in y]
    return x_to_int, y_to_int

In [49]:
def data_preprocessing(data, slide, num_letters, char_to_int):
    x, y = extract_data(data, slide)
    x_to_int, y_to_int = char_to_int_data(x, y, char_to_int)
     # 对X进行独热编码
    x_one_hot = torch.zeros(len(x_to_int), slide, num_letters)
    for i in range(len(x_to_int)):
        for j in range(slide):
            x_one_hot[i, j, x_to_int[i][j]] = 1
    y_tensor = torch.tensor(y_to_int, dtype=torch.long)
    return x_one_hot, y_tensor

In [50]:
X, y = data_preprocessing(data, time_step, num_letters, char_to_int)
print(X.shape)
print(y)

torch.Size([44962, 20, 23])
tensor([ 7,  5, 13,  ...,  4,  9,  8])


# 数据分割

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)

# 定义模型

In [52]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  
        return x


In [53]:
model = LSTMModel(num_letters, 20, num_letters)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型

In [54]:
def train_model(model, data_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in data_loader:
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=1000, shuffle=True)
train_model(model, train_loader, criterion, optimizer)

In [55]:
# 预测函数
def predict(model, data_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for x_batch, _ in data_loader:
            outputs = model(x_batch)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
    return predictions

# 模型评估和预测

In [56]:
# 训练集和测试集预测
train_predictions = predict(model, DataLoader(TensorDataset(X_train, y_train), batch_size=1000))
test_predictions = predict(model, DataLoader(TensorDataset(X_test, y_test), batch_size=1000))

In [57]:
# 计算准确率
train_accuracy = accuracy_score(y_train.numpy(), train_predictions)
test_accuracy = accuracy_score(y_test.numpy(), test_predictions)
print('训练集准确率:', train_accuracy)
print('测试集准确率:', test_accuracy)

训练集准确率: 0.6643766217719016
测试集准确率: 0.6591060707138092


In [58]:
# 新样例预测
new_letters = "flare is a teacher in ai industry. He obtained his phd in Australia."
X_new, _ = data_preprocessing(new_letters, time_step, num_letters, char_to_int)
new_predictions = predict(model, DataLoader(TensorDataset(X_new, torch.zeros(X_new.shape[0], dtype=torch.long)), batch_size=1))
new_predict_char = [int_to_char[i] for i in new_predictions]
print(new_predict_char)

# 显示预测结果
for i in range(len(new_letters) - time_step):
    print(new_letters[i:i + time_step], '--predict next letter is--', new_predict_char[i])

['n', ' ', 'a', 'i', 'i', 'i', 'n', ' ', 'u', 's', 't', 'r', 'y', '.', ' ', 'H', 'e', ' ', 'o', 'u', 't', 'e', 'e', 'e', ' ', ' ', ' ', 'h', 'n', 'n', ' ', 'h', 'h', 'd', ' ', 'i', 'i', ' ', 'A', 'u', 's', 't', 'r', 'r', 'l', 'i', 'i', ' ']
flare is a teacher i --predict next letter is-- n
lare is a teacher in --predict next letter is--  
are is a teacher in  --predict next letter is-- a
re is a teacher in a --predict next letter is-- i
e is a teacher in ai --predict next letter is-- i
 is a teacher in ai  --predict next letter is-- i
is a teacher in ai i --predict next letter is-- n
s a teacher in ai in --predict next letter is--  
 a teacher in ai ind --predict next letter is-- u
a teacher in ai indu --predict next letter is-- s
 teacher in ai indus --predict next letter is-- t
teacher in ai indust --predict next letter is-- r
eacher in ai industr --predict next letter is-- y
acher in ai industry --predict next letter is-- .
cher in ai industry. --predict next letter is--  
her in ai